In [ ]:
'''
    @Author: King
    @Date: 2019.03.17
    @Purpose: 文本分类实战（二）—— textCNN 模型
    @Introduction:  textCNN 模型 文本分类
    @Datasets: cnews 中文文本分类数据集。
    @Link: https://github.com/jiangxinyang227/textClassifier/tree/master/textCNN
    @Reference: https://www.cnblogs.com/jiangxinyang/p/10207482.html
'''

## 数据集介绍

cnews 中文文本分类数据集，其目标是判断一段话的类别，数据预处理如文本分类实战（一）—— word2vec预训练词向量中一样，预处理后的文件为../data/cnews_data/labeledCharTrain.csv。

## textCNN 模型介绍

### 1 引言

将神经网络应用于大图像时，输入可能有上百万个维度，如果输入层和隐含层进行“全连接”，需要训练的参数将会非常多。如果构建一个“部分联通”网络，每个隐含单元仅仅只能连接输入单元的一部分，参数数量会显著下降。卷积神经网络就是基于这个原理而构建的。这其中的思想就是，降维或者说是特征选择，通过前面的卷积层或者池化层将重要的特征选取出来，然后全连接进行分类。特征是最重要的。

###  2. Text-CNN 的引入
从 Yoon Kim 发表的论文《Convolutional Naural Networks for Sentence Classification》{http://link.zhihu.com/?target=https%3A//arxiv.org/abs/1408.5882}中，首次将 CNN 引入文本分类。

论文所提出的模型结构如下图所示：
![textCNN](https://mmbiz.qpic.cn/mmbiz_png/pCFs8q3BZjcIIePZDYyH2IEZcMFUVlVn89qOHVQBlh9cCZmBwUYicic4hGY07GMbR7XN1KvlagHM3FsKFPrQqbGg/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

1. 这里的输入层显示有两个channel，其实我们可以看作是一个，因为后文中说到这两个channel分别是static和non-static，即使用的词向量是否随着训练发生变化。non-static就是词向量随着模型训练变化（Fine tune），这样的好处是词向量可以根据数据集做适当调整，但是CS224d课程里也说过当数据集较小时不推荐此操作，否则容易产生过拟合现象。static就是直接使用word2vec训练好的词向量即可。此外，由图可知，输入层是将一个句子所有单词（padding）的词向量进行拼接成一个矩阵，每一行代表一个词。每个句子固定20个词，如果不够的补padding。


2. 卷积层，不做过多解释。每个卷积核的大小为filter_size*embedding_size。filter_size代表卷积核纵向上包含单词个数，即认为相邻几个词之间有词序关系，代码里使用的是[3，4，5]。embedding_size就是词向量的维数。每个卷积核计算完成之后我们就得到了1个列向量，代表着该卷积核从句子中提取出来的特征。有多少和卷积核就能提取出多少种特征，即图中在纵深方向上channel的数量。

![textCNN](https://mmbiz.qpic.cn/mmbiz_png/pCFs8q3BZjcIIePZDYyH2IEZcMFUVlVneQTsXuIIFOydQTU1JRf2TXf8nfY6lXDKCPqF6f1aKTxzWJdcnD36FQ/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

3. 池化层。文中提到pooling操作就是将卷积得到的列向量的最大值提取出来。这样pooling操作之后我们会获得一个num_filters维的行向量，即将每个卷积核的最大值连接起来。这样做还有一个好处就是，如果我们之前没有对句子进行padding操作，那么句子的长度是不同的，卷积之后得到的列向量维度也是不同的，可以通过pooling来消除句子之间长度不同的差异。

![textCNN](https://mmbiz.qpic.cn/mmbiz_png/pCFs8q3BZjcIIePZDYyH2IEZcMFUVlVnyetjOK1Yy7JK4dYBicNianicooYaaw1IicLDh4901aS8JKdp4WtJhQDvhQ/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

CNN中采用Max Pooling操作有几个好处：

- 第一点，这个操作可以保证特征的位置与旋转不变性，因为不论这个强特征在哪个位置出现，都会不考虑其出现位置而能把它提出来。但是对于NLP来说，这个特性其实并不一定是好事，因为在很多NLP的应用场合，特征的出现位置信息是很重要的，比如主语出现位置一般在句子头，宾语一般出现在句子尾等等。

- 第二个，MaxPooling能减少模型参数数量，有利于减少模型过拟合问题。因为经过Pooling操作后，往往把2D或者1D的数组转换为单一数值，这样对于后续的Convolution层或者全联接隐层来说无疑单个Filter的参数或者隐层神经元个数就减少了。

- 第三个，对于NLP任务来说，可以把变长的输入X整理成固定长度的输入。因为CNN最后往往会接全联接层，而其神经元个数是需要事先定好的，如果输入是不定长的那么很难设计网络结构。


但是，CNN模型采取MaxPooling Over Time也有缺点：

- 第一个：特征的位置信息在这一步骤完全丢失。在卷积层其实是保留了特征的位置信息的，但是通过取唯一的最大值，现在在Pooling层只知道这个最大值是多少，但是其出现位置信息并没有保留；

- 第二个：有时候有些强特征会出现多次，出现次数越多说明这个特征越强，但是因为Max Pooling只保留一个最大值，就是说同一特征的强度信息丢失了。


4. 全连接层，为了将pooling层输出的向量转化为我们想要的预测结果，加上一个softmax层即可。针对电影评价的分类任务，就是将其转化为正面、负面两个结果。文中还提到了过拟合的问题，因为实验中所使用的数据集相对较小，很容易就会发生过拟合现象，在实验过程中也会发现当迭代3000多轮的时候准确率就会接近1。所以这里引如dropout来减少过拟合现象。此外还可以考虑L2正则化等方法实现防止过拟合的功能。

![textCNN](https://mmbiz.qpic.cn/mmbiz_png/pCFs8q3BZjcIIePZDYyH2IEZcMFUVlVn0Z66R1JzzNVClxbxBjKickQMberzjic4ia8RnBT8ticFest5wvfjoIib5hA/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

到这里其实对论文模型的架构已经介绍的差不多了，至于实验结果自然不必说，达到了很好的效果，现在其分类性能也成了新论文的标准线。具体的结果分析可以参考论文，这里不再详细赘述。



论文《A Sensitivity Analysis of (and Practitioners’ Guide to) Convolutional Neural Networks for Sentence Classification》{http://link.zhihu.com/?target=https%3A//arxiv.org/pdf/1510.03820.pdf}专门做了各种控制变量的实验对比。

![textcnn](https://mmbiz.qpic.cn/mmbiz_png/pCFs8q3BZjcIIePZDYyH2IEZcMFUVlVn3nKQicealECDCd6zEDwFPf5awfGIfLmslYDlQEUUJ8ria0fRiaYUo5WIA/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

word embedding的维度是5，对于句子 i like this movie very muc，转换成矩阵AϵR7×5；

有6个卷积核，尺寸为(2×5), (3×5), (4×5)，每种尺寸各2个，A分别与以上卷积核进行卷积操作（这里的Stride Size相当于等于高度h）；

再用激活函数激活，每个卷积核得到了特征向量(feature maps)；

使用1-max pooling提取出每个feature map的最大值；

然后在级联得到最终的特征表达；

将特征输入至softmax layer进行分类, 在这层可以进行正则化操作( l2-regulariation)。

In [3]:
import os
import csv
import time
import datetime
import random
import json

from collections import Counter
from math import sqrt

import gensim
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

d:\progrom\python\python\python3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\progrom\python\python\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
# 配置参数

class TrainingConfig(object):
    epoches = 4
    evaluateEvery = 10
    checkpointEvery = 10
    learningRate = 0.001
    
class ModelConfig(object):
    embeddingSize = 200
    numFilters = 128

    filterSizes = [2, 3, 4, 5]
    dropoutKeepProb = 0.5
    l2RegLambda = 0.0
    
class Config(object):
    sequenceLength = 200  # 取了所有序列长度的均值
    batchSize = 128
    
    dataSource = "../data/cnews_data/labeledCharTrain.csv"
    
    stopWordSource = "../data/cnews_data/stopwords.txt"
    
    numClasses = 10
    
    rate = 0.8  # 训练集的比例
    
    training = TrainingConfig()
    
    model = ModelConfig()

    
# 实例化配置参数对象
config = Config()

## 生成训练数据

　　1）将数据加载进来，将句子分割成词表示，并去除低频词和停用词。

　　2）将词映射成索引表示，构建词汇-索引映射表，并保存成json的数据格式，之后做inference时可以用到。（注意，有的词可能不在word2vec的预训练词向量中，这种词直接用UNK表示）

　　3）从预训练的词向量模型中读取出词向量，作为初始化值输入到模型中。

　　4）将数据集分割成训练集和测试集

In [5]:
# 整块 update
import sys
# 数据预处理的类，生成训练集和测试集

class Dataset(object):
    def __init__(self, config):
        self._dataSource = config.dataSource
        self._stopWordSource = config.stopWordSource  
        
        self._sequenceLength = config.sequenceLength  # 每条输入的序列处理为定长
        self._embeddingSize = config.model.embeddingSize
        self.numClasses = config.numClasses        # update
        self._batchSize = config.batchSize
        self._rate = config.rate
        
        self._stopWordDict = {}
        
        self.trainReviews = []
        self.trainLabels = []
        
        self.evalReviews = []
        self.evalLabels = []
        
        self.wordEmbedding =None
        
        self._wordToIndex = {}
        self._indexToWord = {}
        self.label_to_index = {}
        self.index_to_label = {}
        
    def _readData(self, filePath):
        """
        从csv文件中读取数据集
        """
        
        df = pd.read_csv(filePath)
        labels = df["class"].tolist()                # update
        review = df["review_cut_word"].tolist()
        reviews = [str(line).strip().split() for line in review]

        return reviews, labels

    def _reviewProcess(self, review, sequenceLength, wordToIndex):
        """
        将数据集中的每条评论用index表示
        wordToIndex中“pad”对应的index为0
        """
        
        reviewVec = np.zeros((sequenceLength))
        sequenceLen = sequenceLength
        
        # 判断当前的序列是否小于定义的固定序列长度
        if len(review) < sequenceLength:
            sequenceLen = len(review)
            
        for i in range(sequenceLen):
            if review[i] in wordToIndex:
                reviewVec[i] = wordToIndex[review[i]]
            else:
                reviewVec[i] = wordToIndex["UNK"]

        return reviewVec

    def _labelProcess(self, y, label_to_index):
        """
        将数据集中的每个label用index表示
        label_to_index“pad”对应的index为0
        """
        input_y = []
        for item in y:
            temp = [0] * self.numClasses
            temp[label_to_index[item]] = 1
            input_y.append(temp)
        
        return input_y
    
    def _genTrainEvalData(self, x, y, rate):
        """
        生成训练集和验证集
        """
        
        reviews = []
        labels = []
        
        input_y = self._labelProcess(y,self.label_to_index)
#         print("input_y:{0}".format(input_y[0:2]))
        
#         sys.exit(0)
        
        # 遍历所有的文本，将文本中的词转换成index表示
        for i in range(len(x)):
            reviewVec = self._reviewProcess(x[i], self._sequenceLength, self._wordToIndex)
            reviews.append(reviewVec)
            
            labels.append(input_y[i])
            
        trainIndex = int(len(x) * rate)
        
        trainReviews = np.asarray(reviews[:trainIndex], dtype="int64")
        trainLabels = np.array(labels[:trainIndex], dtype="float32")
        
        evalReviews = np.asarray(reviews[trainIndex:], dtype="int64")
        evalLabels = np.array(labels[trainIndex:], dtype="float32")

        return trainReviews, trainLabels, evalReviews, evalLabels
        
    def _genVocabulary(self, reviews,labels):
        """
        生成词向量和词汇-索引映射字典，可以用全数据集
        """
        
        allWords = [word for review in reviews for word in review]
        
        # 去掉停用词
        subWords = [word for word in allWords if word not in self.stopWordDict]
        
        wordCount = Counter(subWords)  # 统计词频
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        
        # 去除低频词
        words = [item[0] for item in sortWordCount if item[1] >= 5]
        
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding
        
        self._wordToIndex = dict(zip(vocab, list(range(len(vocab)))))
        self._indexToWord = dict(zip(list(range(len(vocab))), vocab))
        

        labels = set(labels)

        for i, label in enumerate(labels):
            self.label_to_index[label] = i
            self.index_to_label[int(i)] = label
        
        
        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open("../data/wordJson/wordToIndex_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._wordToIndex, f)
        
        with open("../data/wordJson/indexToWord_cnews.json", "w", encoding="utf-8") as f:
            json.dump(self._indexToWord, f)
            
        with open('../data/wordJson/label_to_index_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.label_to_index, f, ensure_ascii=False)

        with open('../data/wordJson/index_to_label_cnews.json', 'w', encoding='utf-8') as f:
            json.dump(self.index_to_label, f, ensure_ascii=False)
            
    def _getWordEmbedding(self, words):
        """
        按照我们的数据集中的单词取出预训练好的word2vec中的词向量
        """
        
        wordVec = gensim.models.KeyedVectors.load_word2vec_format("../data/ChnSentiCorp_cn_data/word2Vec.bin", binary=True)
        vocab = []
        wordEmbedding = []
        
        # 添加 "pad" 和 "UNK", 
        vocab.append("pad")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))
        
        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                #print(word + "不存在于词向量中")
                pass
                
        return vocab, np.array(wordEmbedding)
    
    def _readStopWord(self, stopWordPath):
        """
        读取停用词
        """
        
        with open(stopWordPath, "r", encoding="utf-8") as f:
            stopWords = f.read()
            stopWordList = stopWords.splitlines()
            # 将停用词用列表的形式生成，之后查找停用词时会比较快
            self.stopWordDict = dict(zip(stopWordList, list(range(len(stopWordList)))))
            
    def dataGen(self):
        """
        初始化训练集和验证集
        """
        
        # 初始化停用词
        self._readStopWord(self._stopWordSource)
        
        # 初始化数据集
        reviews, labels = self._readData(self._dataSource)
        
        # 初始化词汇-索引映射表和词向量矩阵
        self._genVocabulary(reviews,labels)
        
        # 初始化训练集和测试集
        trainReviews, trainLabels, evalReviews, evalLabels = self._genTrainEvalData(reviews, labels, self._rate)
        self.trainReviews = trainReviews
        self.trainLabels = trainLabels
        print("self.trainReviews[0:2]:{0}".format(self.trainReviews[0:3]))
        print("self.trainLabels[0:2]:{0}".format(self.trainLabels[0:3]))
        
        self.evalReviews = evalReviews
        self.evalLabels = evalLabels
        print("self.evalReviews[0:2]:{0}".format(self.evalReviews[0:3]))
        print("self.evalLabels[0:2]:{0}".format(self.evalLabels[0:3]))
        
        
data = Dataset(config)
data.dataGen()

d:\progrom\python\python\python3\lib\site-packages\ipykernel_launcher.py:167: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


self.trainReviews[0:2]:[[   1    1    1    1    1    1    1    1    1    1    1    1 1131    1
     1    1    1  478  142    1   27    2    1  483    1    1    1    1
     1    1 1281    1    1    1    1    1    1  886    1    1    1    1
     1    1    1    1    1    1 1005    1    1    1   13    1    1    1
     1    1    1    1    1    1    1  203  215    1    1    1    1    1
     1    1   27    1    1    1    1    1  102    1  152    1    1    1
     1  451    1    1  753    1    1  226    1    1   65    1    1    1
     1    1    1   19    1    1    1    1    1  849    1    1 2243    1
     1    1    1    1    1    1    1    1  878    1    1    1    1    1
     1    1    1    1    1    1    1    1    1    1    1  419    1    1
   196    1    1    1    1    1    1    1    1    1    1    1    1    1
     1    1   10    1    1    8    1    1 1654    1    1    1    1    1
     1    1    1   36    1    1    1 1151    1    1    1    1    1    1
     1    1  120    1    1 1162    1    1

In [6]:
print("train data shape: {}".format(data.trainReviews.shape))
print("train label shape: {}".format(data.trainLabels.shape))
print("eval data shape: {}".format(data.evalReviews.shape))

train data shape: (3999, 200)
train label shape: (3999, 10)
eval data shape: (1000, 200)


In [7]:
# 输出batch数据集

def nextBatch(x, y, batchSize):
        """
        生成batch数据集，用生成器的方式输出
        """
    
        perm = np.arange(len(x))
        np.random.shuffle(perm)
        x = x[perm]
        y = y[perm]
        
        numBatches = len(x) // batchSize

        for i in range(numBatches):
            start = i * batchSize
            end = start + batchSize
            batchX = np.array(x[start: end], dtype="int64")
            batchY = np.array(y[start: end], dtype="float32")
            
            yield batchX, batchY

In [22]:
# 构建模型
class TextCNN(object):
    """
    Text CNN 用于文本分类
    """
    def __init__(self, config, wordEmbedding):

        # 定义模型的输入
        self.numClasses = config.numClasses      # update
        self.inputX = tf.placeholder(tf.int32, [None, config.sequenceLength], name="inputX")
        self.inputY = tf.placeholder(tf.float32, [None, self.numClasses], name="inputY")
        
        self.dropoutKeepProb = tf.placeholder(tf.float32, name="dropoutKeepProb")
        
        # 定义l2损失
        l2Loss = tf.constant(0.0)
        
        # 词嵌入层
        with tf.name_scope("embedding"):

            # 利用预训练的词向量初始化词嵌入矩阵
            self.W = tf.Variable(tf.cast(wordEmbedding, dtype=tf.float32, name="word2vec") ,name="W")
            # 利用词嵌入矩阵将输入的数据中的词转换成词向量，维度[batch_size, sequence_length, embedding_size]
            self.embeddedWords = tf.nn.embedding_lookup(self.W, self.inputX)
            # 卷积的输入是思维[batch_size, width, height, channel]，因此需要增加维度，用tf.expand_dims来增大维度
            self.embeddedWordsExpanded = tf.expand_dims(self.embeddedWords, -1)

        # 创建卷积和池化层
        pooledOutputs = []
        # 有三种size的filter，3， 4， 5，textCNN是个多通道单层卷积的模型，可以看作三个单层的卷积模型的融合
        for i, filterSize in enumerate(config.model.filterSizes):
            with tf.name_scope("conv-maxpool-%s" % filterSize):
                # 卷积层，卷积核尺寸为filterSize * embeddingSize，卷积核的个数为numFilters
                # 初始化权重矩阵和偏置
                filterShape = [filterSize, config.model.embeddingSize, 1, config.model.numFilters]
                W = tf.Variable(tf.truncated_normal(filterShape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[config.model.numFilters]), name="b")
                conv = tf.nn.conv2d(
                    self.embeddedWordsExpanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                
                # relu函数的非线性映射
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                
                # 池化层，最大池化，池化是对卷积后的序列取一个最大值
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, config.sequenceLength - filterSize + 1, 1, 1],  # ksize shape: [batch, height, width, channels]
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooledOutputs.append(pooled)  # 将三种size的filter的输出一起加入到列表中

        # 得到CNN网络的输出长度
        numFiltersTotal = config.model.numFilters * len(config.model.filterSizes)
        
        # 池化后的维度不变，按照最后的维度channel来concat
        self.hPool = tf.concat(pooledOutputs, 3)
        
        # 摊平成二维的数据输入到全连接层
        self.hPoolFlat = tf.reshape(self.hPool, [-1, numFiltersTotal])

        # dropout
        with tf.name_scope("dropout"):
            self.hDrop = tf.nn.dropout(self.hPoolFlat, self.dropoutKeepProb)
       
        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW",
                shape=[numFiltersTotal, self.numClasses],
                initializer=tf.contrib.layers.xavier_initializer())
            outputB= tf.Variable(tf.constant(0.1, shape=[self.numClasses]), name="outputB")
            l2Loss += tf.nn.l2_loss(outputW)
            l2Loss += tf.nn.l2_loss(outputB)
            self.scores = tf.nn.xw_plus_b(self.hDrop, outputW, outputB, name='scores')
            self.predictions = tf.argmax(self.scores, 1, name='predictions')
        
        # 计算二元交叉熵损失
        with tf.name_scope("loss"):
            # update
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.inputY)
            self.loss = tf.reduce_mean(losses) + config.model.l2RegLambda * l2Loss
            
        # accuracy
        with tf.name_scope('accuracy'):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.inputY, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')

In [23]:
# 定义性能指标函数

def mean(item):
    return sum(item) / len(item)


def genMetrics(trueY, predY, binaryPredY):
    """
    生成acc和auc值
    """
    print("trueY:{0}".format(trueY))
    print("predY:{0}".format(predY))
    print("binaryPredY:{0}".format(binaryPredY))
    #auc = roc_auc_score(trueY, predY)
    accuracy = accuracy_score(trueY, binaryPredY)
    precision = precision_score(trueY, binaryPredY)
    recall = recall_score(trueY, binaryPredY)
    
    return round(accuracy, 4), round(auc, 4), round(precision, 4), round(recall, 4)

In [24]:
# 训练模型
import os
import shutil
# 生成训练集和验证集
trainReviews = data.trainReviews
trainLabels = data.trainLabels
evalReviews = data.evalReviews
evalLabels = data.evalLabels

wordEmbedding = data.wordEmbedding

# 定义计算图
with tf.Graph().as_default():

    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
    session_conf.gpu_options.allow_growth=True
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.9  # 配置gpu占用率  

    sess = tf.Session(config=session_conf)
    
    # 定义会话
    with sess.as_default():
        cnn = TextCNN(config, wordEmbedding)
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(config.training.learningRate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(cnn.loss)
        # 将梯度应用到变量下，生成训练器
        trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)
        
        # 用summary绘制tensorBoard
        gradSummaries = []
        for g, v in gradsAndVars:
            if g is not None:
                tf.summary.histogram("{}/grad/hist".format(v.name), g)
                tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
        
        outDir = os.path.abspath(os.path.join(os.path.curdir, "summarys"))
        print("Writing to {}\n".format(outDir))
        
        lossSummary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar('accuracy', cnn.accuracy)
        summaryOp = tf.summary.merge_all()
        
        trainSummaryDir = os.path.join(outDir, "train")
        trainSummaryWriter = tf.summary.FileWriter(trainSummaryDir, sess.graph)
        
        evalSummaryDir = os.path.join(outDir, "eval")
        evalSummaryWriter = tf.summary.FileWriter(evalSummaryDir, sess.graph)
        
        
        # 初始化所有变量
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)
        
        # 保存模型的一种方式，保存为pb文件
        shutil.rmtree('../model/textCNN/savedModel_cnews')  # 去除 model 目录下文件
        builder = tf.saved_model.builder.SavedModelBuilder("../model/textCNN/savedModel_cnews")
        sess.run(tf.global_variables_initializer())

        def trainStep(batchX, batchY):
            """
            训练函数
            """   
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: config.model.dropoutKeepProb
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.accuracy],
                feed_dict=feed_dict
            )
            timeStr = datetime.datetime.now().isoformat()
#             acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            print("{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            trainSummaryWriter.add_summary(summaries, step)

        def devStep(batchX, batchY, writer=None):
            """
            验证函数
            """
            feed_dict = {
              cnn.inputX: batchX,
              cnn.inputY: batchY,
              cnn.dropoutKeepProb: 1.0
            }
            _, summaries,step, loss, accuracy = sess.run(
                [trainOp, summaryOp, globalStep, cnn.loss, cnn.accuracy],
                feed_dict=feed_dict
            )
            
            #acc, auc, precision, recall = genMetrics(batchY, predictions, binaryPreds)
            timeStr = datetime.datetime.now().isoformat()
            print("Evaluation:{}: step {}, loss {:g}, acc {:g}".format(timeStr, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
        
        for i in range(config.training.epoches):
            # 训练模型
            print("start training model")
            for batchTrain in nextBatch(trainReviews, trainLabels, config.batchSize):
                trainStep(batchTrain[0], batchTrain[1])

                currentStep = tf.train.global_step(sess, globalStep) 
                if currentStep % config.training.evaluateEvery == 0:
                    print("\nEvaluation:")
                    
                    
                    for batchEval in nextBatch(evalReviews, evalLabels, config.batchSize):
                        devStep(batchEval[0], batchEval[1], writer=evalSummaryWriter)
                        
                        
                if currentStep % config.training.checkpointEvery == 0:
                    # 保存模型的另一种方法，保存checkpoint文件
                    path = saver.save(sess, "../model/textCNN/model/my-model_cnews", global_step=currentStep)
                    print("Saved model checkpoint to {}\n".format(path))
                    
#         inputs = {"inputX": tf.saved_model.utils.build_tensor_info(cnn.inputX),
#                   "keepProb": tf.saved_model.utils.build_tensor_info(cnn.dropoutKeepProb)}

#         outputs = {"binaryPreds": tf.saved_model.utils.build_tensor_info(cnn.binaryPreds)}

#         prediction_signature = tf.saved_model.signature_def_utils.build_signature_def(inputs=inputs, outputs=outputs,
#                                                                                       method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
#         legacy_init_op = tf.group(tf.tables_initializer(), name="legacy_init_op")
#         builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING],
#                                             signature_def_map={"predict": prediction_signature}, legacy_init_op=legacy_init_op)

#         builder.save()

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/hist is illegal; using conv-maxpool-2/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/W:0/grad/sparsity is illegal; using conv-maxpool-2/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/hist is illegal; using conv-maxpool-2/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-2/b:0/grad/sparsity is illegal; using conv-maxpool-2/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using 

KeyboardInterrupt: 